# Can we webscrape GotW?

## import the necessary modules

In [1]:
from bs4 import BeautifulSoup as Soup
from datetime import datetime
import lxml
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
import time
from urllib import request
from webdriver_manager.microsoft import EdgeChromiumDriverManager

## setup

In [2]:
#set the driver
driver = webdriver.Edge(service = Service(EdgeChromiumDriverManager().install()))

#set url
start_url = "https://gow.epsrc.ukri.org/NGBOFindPanels.aspx"

#load page
driver.get(start_url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 101.0.1210
[WDM] - Get LATEST edgedriver version for 101.0.1210 Edge
[WDM] - Driver [C:\Users\sj294\.wdm\drivers\edgedriver\win64\101.0.1210.53\msedgedriver.exe] found in cache


## search settings
The GotW webpage runs on a search basis, with no results returned initially. We want to try and identify all panels at once, so we want to set the search settings as far back as we can go, even if that means we pick up some data we don't really want.

### generating the table of panels

In [3]:
#first step is to set to search between dates - the ui defaults to 'in the last' 6 months
#from inspecting the webpage, the following should work
driver.find_element(by = By.XPATH, value = '//*[@id="oplDates_0"]').click()

#then we need to pick the first day, month, year
#these xpaths were identified by inspecting the UI and interacting with the relevant elements
day_xpath = '//*[@id="oUcStartDate_ddlDay"]/option[1]'
mon_xpath = '//*[@id="oUcStartDate_ddlMonth"]/option[1]'
yr_xpath = '//*[@id="oUcStartDate_ddlYear"]/option[1]'

#now we have to click on those options
driver.find_element(by = By.XPATH, value = '//*[@id="oUcStartDate_ddlDay"]').click() # open day slicer
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, day_xpath))).click() # choose the day

driver.find_element(by = By.XPATH, value = '//*[@id="oUcStartDate_ddlMonth"]').click() # open day slicer
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, mon_xpath))).click() # choose the month

driver.find_element(by = By.XPATH, value = '//*[@id="oUcStartDate_ddlYear"]').click() # open day slicer
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, yr_xpath))).click() # choose the year

#thankfully the webform defaults the 'to' date to the day of use, so no need to update!

#so now all that's left to do is search (on testing, this works!)
driver.find_element(by = By.XPATH, value = '//*[@id="btnSearch"]').click()

## now to grab links to specific panel pages
We've got the table with all the panel results. GotW is set up so each has unique ID, which then defines the URL for the page with the info we're really after. So now we want to go through this list and extract the ids.

In [4]:
#get the page as a soup object
page = Soup(driver.page_source, features = "html.parser")

#from inspecting the page we can see the particular table of interest has id = "dgDetails"
tbl = page.find(id = "dgDetails")

#find all  the 'a' (hyperlink) elements
hyperlinks = tbl.find_all("a", href = True)

#extract the href arguments
hrefs = [hyperlinks[h]["href"] for h in range(len(hyperlinks))]

#now extract the panle ids - exclude the first two rows as these part of the table header
panel_ids = [hrefs[h].split("PanelId=")[1] for h in range(2, len(hrefs), 1)]

#identify if 'view panel', 'prior panel' or 'NGBO panel' - affects final URL
types = []
for h in range(2, len(hrefs), 1):
    if hrefs[h][:4] == "NGBO":
        type = "NGBO"
        types.append(type)
    elif hrefs[h][4:6] == "Pr": 
        type = "Prior"
        types.append(type)
    else:
        type = "View"
        types.append(type)

#finally, turn these into urls
panel_urls = []
for i in range(len(panel_ids)):
    if types[i] == "NGBO":
        url = f"https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId={panel_ids[i]}"
        panel_urls.append(url)
    elif types[i] == "Prior":
        url = f"https://gow.epsrc.ukri.org/ViewPriorPanel.aspx?PanelId={panel_ids[i]}"
        panel_urls.append(url)
    else:
        url = f"https://gow.epsrc.ukri.org/ViewPanel.aspx?PanelId={panel_ids[i]}"
        panel_urls.append(url)
        
#at this point I don't think the driver is needed any more, so we can quit
driver.quit()

#take a look
panel_urls[:10]


['https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId=1-3S0O1',
 'https://gow.epsrc.ukri.org/ViewPanel.aspx?PanelId=5055',
 'https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId=1-PI0M9',
 'https://gow.epsrc.ukri.org/ViewPanel.aspx?PanelId=4590',
 'https://gow.epsrc.ukri.org/ViewPanel.aspx?PanelId=4648',
 'https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId=1-TM70D',
 'https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId=1-1G4FFB',
 'https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId=1-1D9T5E',
 'https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId=1-5T51YR',
 'https://gow.epsrc.ukri.org/NGBOViewPanel.aspx?PanelId=1-5ZF065']

## now start scraping data from the individual panel pages

### grab metadata

In [5]:
#panel page source
panel_source = request.urlopen(panel_urls[0])

#panel page
panel_page = Soup(panel_source, features = "html.parser")

#get the panel name
panel_name = panel_page.find(id = "lblPanelName").text.strip()

#get the meeting date
meeting_date = panel_page.find(id = "lblDateOfPanel").text
meeting_date = datetime.strptime(meeting_date, '%d %B %Y')

### grab table data

In [6]:
#read tables to dataframe
tables = pd.read_html(panel_urls[0], match = "Funding Priority List") #we're using the first one as an example here

tbl_list = [
    tables[0], # by number
    tables[1] # by value
]

#take a look
tbl_list[0]

,0,1,2,3,4,5
0,Funding Priority List,Funded,Unfunded,Referred to a later panel,Decision still awaited,Funding rate(% of grant number)
1,Standard,3,5,0,0,37
2,Including:,Including:,Including:,Including:,Including:,Including:
3,Main List,3,5,0,0,37
4,Please click on relevant Funding Priority List...,Please click on relevant Funding Priority List...,Please click on relevant Funding Priority List...,Please click on relevant Funding Priority List...,Please click on relevant Funding Priority List...,Please click on relevant Funding Priority List...


### function tidy up tables

In [7]:
def tidy_tables(df):
    
    #remove trailing column if there is one
    if(df.shape[1]) == 7:
        df = df.drop(df.columns[6], axis = 1)
    
    #headers - replace colnames with values from first (0th) row
    df = df.rename(columns = df.iloc[0])
    
    #function to tidy colnames
    def tidy_columns(c):
        stripped = c.strip()
        lower = stripped.lower()
        nospace = lower.replace(" ", "_")
        nopct = nospace.replace("%", "pct")
        nochr = nopct.translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=+"})
        return nochr
   
    df.columns = [tidy_columns(c) for c in df.columns.values.tolist()]
    
    #remove unncessary rows
    df = df.loc[df["funding_priority_list"] == "Standard"]
    
    #add columns for panel, meeting date and year
    df["panel"] = panel_name
    df["year"] = meeting_date.year
    df["month"] = meeting_date.month
    df["day"] = meeting_date.day
    
    #rename the funding rate column for conistency
    df = df.rename(columns = {df.columns[5]: "success_rate"})
    
    return df

### run for both the by number and by value tables

In [8]:
#list for tidied dfs
tidied_dfs = [tidy_tables(df) for df in tbl_list]

#concat
tidy_df = pd.concat(tidied_dfs)

#add on data type
tidy_df["data_type"] = ["number", "value"]

tidy_df


,funding_priority_list,funded,unfunded,referred_to_a_later_panel,decision_still_awaited,success_rate,panel,year,month,day,data_type
1,Standard,3,5,0,0,37,Basic Technology CDT Lite,2011,3,15,number
1,Standard,5911461,10051301,0,0,37,Basic Technology CDT Lite,2011,3,15,value
